## Calculate Phi vertical mean for a particular layer and LME

In [1]:
import metabolic as mi
import xarray as xr
import numpy as np
import netCDF4
import os
import pandas as pd
import utils
import warnings
warnings.filterwarnings('ignore')
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.pyplot as plt

In [2]:
USER = os.environ['USER']
path = f'/glade/scratch/{USER}/DPLE-results'

In [3]:
filename = f'{path}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_monthly.nc'
dset = xr.open_dataset(filename)
dz = dset['dz']/1e2 # m
darea = dset['TAREA']/1e10 #km*km
dvol = dz*darea

In [4]:
KMT = dset['KMT']
nlat = dset.dims['nlat']
nlon = dset.dims['nlon']
for xx in range(nlat):
    for yy in range(nlon):
        kk = np.int(KMT[xx, yy])
        if kk < 60:
            dvol[kk:, xx, yy] = float("nan")

In [54]:
varr = 'O2'
#varr = 'T'
#varr = 'S'
#varr = 'AOU'

In [6]:
#filename = f'{path}/DPLE-FOSI_hindcast_Phi_1948-2017_monthly.nc'
#filename = f'{path}/DPLE-FOSI_hindcast_Phi_Eo_space_1948-2017_monthly.nc'
filename = f'{path}/DPLE-FOSI_hindcast_Phi_components_{varr}_1948-2017_monthly.nc'
ds = xr.open_dataset(filename)
dsf = xr.Dataset()
for ii in range(3):
    print(ii)
    Phi_i = ds[f'Phi_{varr}_{ii}']*dvol
    # select a vertical layer
    dsf[f'Phi_{varr}_{ii}_0-200m'] = Phi_i.isel(z_t=range(0, 20, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(0, 20, 1)).sum(dim='z_t'))
    dsf[f'Phi_{varr}_{ii}_200-600m'] = Phi_i.isel(z_t=range(20, 35, 1)).sum(dim='z_t')/(dvol.isel(z_t=range(20, 35, 1)).sum(dim='z_t'))

0
1
2


In [7]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:             (nlat: 384, nlon: 320, time: 840)
Coordinates:
    TLAT                (nlat, nlon) float64 -79.22 -79.22 ... 72.19 72.19
    TLONG               (nlat, nlon) float64 320.6 321.7 322.8 ... 319.4 319.8
    ULAT                (nlat, nlon) float64 -78.95 -78.95 ... 72.41 72.41
    ULONG               (nlat, nlon) float64 321.1 322.3 323.4 ... 319.6 320.0
  * time                (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:0...
Dimensions without coordinates: nlat, nlon
Data variables:
    Phi_AOU_0_0-200m    (nlat, nlon, time) float64 nan nan nan ... nan nan nan
    Phi_AOU_0_200-600m  (nlat, nlon, time) float64 nan nan nan ... nan nan nan
    Phi_AOU_1_0-200m    (nlat, nlon, time) float64 nan nan nan ... nan nan nan
    Phi_AOU_1_200-600m  (nlat, nlon, time) float64 nan nan nan ... nan nan nan
    Phi_AOU_2_0-200m    (nlat, nlon, time) float64 nan nan nan ... nan nan nan
    Phi_AOU_2_200-600m  (nlat, nlon, time) float64 nan nan nan ... nan nan nan

In [8]:
%time
#dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_vertical_mean_1948-2017_monthly.nc', mode='w')
#dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_Eo_space_vertical_mean_1948-2017_monthly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_components_{varr}_vertical_mean_1948-2017_monthly.nc', mode='w')

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.72 µs


## calculate LME-averaged Phi 

In [55]:
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE-FOSI_hindcast_Phi_components_{varr}_vertical_mean_1948-2017_monthly.nc'
dsf = xr.open_dataset(filename)

In [56]:
datadir = f'/glade/u/home/{USER}/O2prediction/Prediction/notebooks/data'
filename = '%s/LME-mask-POP_gx1v6.nc' % datadir
ds1 = xr.open_dataset(filename)
lme_mask = ds1['lme_mask']

In [57]:
#lme = range(np.int(lme_mask.min()), np.int(lme_mask.max()), 1)
lme = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 65]
nmon = dsf.dims['time']
#nmon = dsf.dims['year']
nmon

840

In [58]:
boxPhi = xr.Dataset()
for ii in range(3):
    print(ii)
    phi_box1 = np.ma.zeros([len(lme), nmon])
    phi_box1.mask = True
    phi_box2 = phi_box1.copy()
    for box,hh in zip(lme, range(len(lme))):
        latloc, lonloc = np.where(lme_mask == box)
        ar1 = np.ma.zeros([len(latloc),nmon]);ar1.mask=True
        ar2 = ar1.copy()
        for bb in range(len(latloc)):
            ar1[bb] = dsf[f'Phi_{varr}_{ii}_0-200m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
            ar2[bb] = dsf[f'Phi_{varr}_{ii}_200-600m'].isel(nlat=latloc[bb], nlon=lonloc[bb])
        phi_box1[hh] = np.nanmean(ar1,axis=0)
        phi_box2[hh] = np.nanmean(ar2,axis=0)
#
    boxPhi[f'Phi_{varr}_{ii}_0-200m'] = xr.DataArray(data=phi_box1, coords=dict(lme=('lme', lme), time=('time', dsf.coords['time'].values)), dims=('lme','time'))
    boxPhi[f'Phi_{varr}_{ii}_200-600m'] = xr.DataArray(data=phi_box2, coords=dict(lme=('lme', lme), time=('time', dsf.coords['time'].values)), dims=('lme','time'))
#    boxPhi[f'Phi_{varr}_{ii}_0-200m'] = xr.DataArray(data=phi_box1, coords=dict(lme=('lme', lme), time=('year', dsf.coords['year'].values)), dims=('lme','year'))
#    boxPhi[f'Phi_{varr}_{ii}_200-600m'] = xr.DataArray(data=phi_box2, coords=dict(lme=('lme', lme), time=('year', dsf.coords['year'].values)), dims=('lme','year'))

0
1
2


In [59]:
boxPhi.load()

<xarray.Dataset>
Dimensions:            (lme: 11, time: 840)
Coordinates:
  * lme                (lme) int64 1 2 3 4 5 6 7 8 9 10 65
  * time               (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
Data variables:
    Phi_O2_0_0-200m    (lme, time) float64 0.03624 0.0697 ... -0.06437 -0.02915
    Phi_O2_0_200-600m  (lme, time) float64 0.01347 0.0144 ... -0.0103 -0.01186
    Phi_O2_1_0-200m    (lme, time) float64 0.04736 0.09004 ... -0.07969 -0.03597
    Phi_O2_1_200-600m  (lme, time) float64 0.01669 0.01784 ... -0.01271 -0.0146
    Phi_O2_2_0-200m    (lme, time) float64 0.07258 0.1356 ... -0.112 -0.05033
    Phi_O2_2_200-600m  (lme, time) float64 0.02349 0.02508 ... -0.01774 -0.02032

In [60]:
dout = f'/glade/scratch/{USER}/DPLE-results'
#boxPhi.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_LME_1948-2017_monthly.nc', mode='w')
#boxPhi.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_Eo_space_LME_1948-2017_monthly.nc', mode='w')
boxPhi.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_components_{varr}_LME_1948-2017_monthly.nc', mode='w')
#boxPhi.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_components_{varr}_LME_1948-2017_monthly_to_yearly.nc', mode='w')